<a href="https://colab.research.google.com/github/farfaness/Spark/blob/master/Pyspark_Databricks_Community_Edition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Pour la doc :
https://spark.apache.org/docs/2.2.0/api/python/pyspark.html

Normallement commancer par instancier un spark context. 
conf = SparkConf().setAppName(appName).setMaster(master)
sc = SparkContext(conf=conf)

#### Création d'un RDD contenant des nombres de 0 à 499. Notez bien le type d'objet manipulé (RDD -> Distribué)

In [0]:
data = list(range(0, 500))
distData = sc.parallelize(data)

#### Affichage du contenu et de la taille

In [0]:
# Afficher le contenu (les 5 premières lignes)
print(distData.take(5)) #ou on peut utiliser collect pour print tout le RDD

# Afficher la taille
print(distData.count())

[0, 1, 2, 3, 4]
500

#### Afficher le nombre de partition du RDD (le réduire à 2 puis le mettre à 4)

In [0]:
# Afficher le nbr de partitions
print(distData.getNumPartitions())

# Reduce to 2 partitions
rdd2 = distData.coalesce(2)
print(rdd2.getNumPartitions())

# Increase to 4 partitions
rdd4 = rdd2.repartition(4)
print(rdd4.getNumPartitions())

8
2
4

### Quelle est la difference entre coalesce et repartition ?

-> Coalesce uses existing partitions to minimize the amount of data that's shuffled. 

-> Repartition creates new partitions and does a full shuffle. 

Coalesce results in partitions with different amounts of data (sometimes partitions that have much different sizes) and repartition results in roughly equal sized partitions.

**Is coalesce or repartition faster?**

coalesce may run faster than repartition, but unequal sized partitions are generally slower to work with than equal sized partitions. You'll usually need to repartition datasets after filtering a large data set. I've found repartition to be faster overall because Spark is built to work with equal sized partitions.

source : https://stackoverflow.com/questions/31610971/spark-repartition-vs-coalesce

#### Récupérer un échantillon de données (disons 100 données, prises au hasard)

In [0]:
# Deux méthodes : une plus exacte mais exige que l'échantillon soit petit
# L'autre plus aléatoire mais supporte une très grande taille pour l'échantillon
print(len(range_rdd.takeSample(True, 100))) 
print(range_rdd.sample(True, 0.2).count())

[10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407]

Difference entre les deux méthodes?

-> takesample = Prend nbr en fonction d'un tirage aléatoire d'index (mais necessite d'avoir tout les éléments sur lesquels faire le tirage, du coup fonctionne que sur les petits échantillons, car mets tout dans la mémoire du driver, donc pas distribué, pas big data)

-> sample = big data, pour chaqu'un des nombres, fait un tirage de type bernouilli ou poisson (avec la proba qu'on lui donne qui correspondant à la proba que la pièce tombe sur 1; par example 2/5)


### Quelle est la différence entre Take et TakeOrdered ? 



The take action returns an array of the first n elements (not ordered) whereas takeOrdered returns an array with the first n elements after a sort

In [0]:
print(rdd4.takeOrdered(100))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]

#### Différence entre Map et FlatMap ?

Flatmap : Return a new RDD by first applying a function to all elements of this RDD, and then flattening the results

In [0]:
# Map example
print(sc.parallelize(range(1000)).map(lambda x: 2 * x).take(10))

# Flatmap example
lines = sc.parallelize("coucou")
print(lines.flatMap(lambda line: line.split(' ')).collect())

[0, 2, 4, 6, 8, 10, 12, 14, 16, 18]
['c', 'o', 'u', 'c', 'o', 'u']

#### Enregistrer votre RDD dans HDFS (au format csv)

In [0]:
df.rdd.saveAsTextFile("df2.csv")

#### Liser le fichier enregistré précédemment (avec spark.sparkContext.textFile ET avec spark.sparkContext.wholeTextFiles). Quelle est la différence entre ces deux primitives ?

In [0]:
# With TextFile
lines = sc.textFile("df2.csv")
print(lines.take(5))

print("%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%")

# With whole textFiles
lines2 = sc.wholeTextFiles("df2.csv")
print(lines2.take(5))


['Row(value=10)', 'Row(value=11)', 'Row(value=12)', 'Row(value=13)', 'Row(value=14)']
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%
[('dbfs:/df2/part-00000', 'Row(value=10)\nRow(value=11)\nRow(value=12)\nRow(value=13)\nRow(value=14)\nRow(value=15)\nRow(value=16)\nRow(value=17)\nRow(value=18)\nRow(value=19)\nRow(value=50)\nRow(value=51)\nRow(value=52)\nRow(value=53)\nRow(value=54)\nRow(value=55)\nRow(value=56)\nRow(value=57)\nRow(value=58)\nRow(value=59)\nRow(value=90)\nRow(value=91)\nRow(value=92)\nRow(value=93)\nRow(value=94)\nRow(value=95)\nRow(value=96)\nRow(value=97)\nRow(value=98)\nRow(value=99)\nRow(value=130)\nRow(value=131)\nRow(value=132)\nRow(value=133)\nRow(value=134)\nRow(value=135)\nRow(value=136)\nRow(value=137)\nRow(value=138)\nRow(value=139)\nRow(value=170)\nRow(value=171)\nRow(value=172)\nRow(value=173)\nRow(value=174)\nRow(value=175)\nRow(value=176)\nRow(value=177)\nRow(value=178)\nRow(value=179)\nRow(value=210)\nRow(value=211)\nRow(value=212)\nRow(value=213)\nRow(value=214)\nRow(value=215)\nRow(value=216)\nRow(value=217)\nRow(value=218)\nRow(value=219)\nRow(value=278)\nRow(value=279)\n'), ('dbfs:/df2/part-00001', 'Row(value=280)\nRow(value=281)\nRow(value=282)\nRow(value=283)\nRow(value=284)\nRow(value=285)\nRow(value=286)\nRow(value=287)\nRow(value=318)\nRow(value=319)\nRow(value=320)\nRow(value=321)\nRow(value=322)\nRow(value=323)\nRow(value=324)\nRow(value=325)\nRow(value=326)\nRow(value=327)\nRow(value=358)\nRow(value=359)\nRow(value=360)\nRow(value=361)\nRow(value=362)\nRow(value=363)\nRow(value=364)\nRow(value=365)\nRow(value=366)\nRow(value=367)\nRow(value=398)\nRow(value=399)\nRow(value=400)\nRow(value=401)\nRow(value=402)\nRow(value=403)\nRow(value=404)\nRow(value=405)\nRow(value=406)\nRow(value=407)\nRow(value=438)\nRow(value=439)\nRow(value=440)\nRow(value=441)\nRow(value=442)\nRow(value=443)\nRow(value=444)\nRow(value=445)\nRow(value=446)\nRow(value=447)\nRow(value=478)\nRow(value=479)\nRow(value=480)\nRow(value=481)\nRow(value=482)\nRow(value=483)\nRow(value=484)\nRow(value=485)\nRow(value=486)\nRow(value=487)\nRow(value=20)\nRow(value=21)\nRow(value=22)\nRow(value=23)\n'), ('dbfs:/df2/part-00002', 'Row(value=24)\nRow(value=25)\nRow(value=26)\nRow(value=27)\nRow(value=28)\nRow(value=29)\nRow(value=60)\nRow(value=61)\nRow(value=62)\nRow(value=63)\nRow(value=64)\nRow(value=65)\nRow(value=66)\nRow(value=67)\nRow(value=68)\nRow(value=69)\nRow(value=100)\nRow(value=101)\nRow(value=102)\nRow(value=103)\nRow(value=104)\nRow(value=105)\nRow(value=106)\nRow(value=107)\nRow(value=108)\nRow(value=109)\nRow(value=140)\nRow(value=141)\nRow(value=142)\nRow(value=143)\nRow(value=144)\nRow(value=145)\nRow(value=146)\nRow(value=147)\nRow(value=148)\nRow(value=149)\nRow(value=180)\nRow(value=181)\nRow(value=182)\nRow(value=183)\nRow(value=184)\nRow(value=185)\nRow(value=186)\nRow(value=187)\nRow(value=188)\nRow(value=189)\nRow(value=220)\nRow(value=221)\nRow(value=222)\nRow(value=223)\nRow(value=224)\nRow(value=225)\nRow(value=226)\nRow(value=227)\nRow(value=228)\nRow(value=229)\nRow(value=248)\nRow(value=249)\nRow(value=250)\nRow(value=251)\nRow(value=252)\nRow(value=253)\n'), ('dbfs:/df2/part-00003', 'Row(value=254)\nRow(value=255)\nRow(value=256)\nRow(value=257)\nRow(value=288)\nRow(value=289)\nRow(value=290)\nRow(value=291)\nRow(value=292)\nRow(value=293)\nRow(value=294)\nRow(value=295)\nRow(value=296)\nRow(value=297)\nRow(value=328)\nRow(value=329)\nRow(value=330)\nRow(value=331)\nRow(value=332)\nRow(value=333)\nRow(value=334)\nRow(value=335)\nRow(value=336)\nRow(value=337)\nRow(value=368)\nRow(value=369)\nRow(value=370)\nRow(value=371)\nRow(value=372)\nRow(value=373)\nRow(value=374)\nRow(value=375)\nRow(value=376)\nRow(value=377)\nRow(value=408)\nRow(value=409)\nRow(value=410)\nRow(value=411)\nRow(value=412)\nRow(value=413)\nRow(value=414)\nRow(value=415)\nRow(value=416)\nRow(value=417)\nRow(value=448)\nRow(value=449)\nRow(value=450)\nRow(value=451)\nRow(value=452)\nRow(value=453)\nRow(value=454)\nRow(value=455)\nRow(va